In [37]:
import pandas as pd
import numpy as np

# Importing dataset


In [38]:
# !kaggle datasets download A-d notshrirang/spotify-million-song-dataset

In [39]:
# %unzip spotify-million-song-dataset.zip

In [40]:
spotifyDF = pd.read_csv("/kaggle/input/smalldataset/spotifyDFMS3.csv")

# Milestone 3

In [41]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [43]:
spotifyDFMS3=spotifyDF.copy()

In [44]:
def split_sentences(df, col_name):
    df['X'] = df[col_name].apply(lambda x: ' '.join(x.split()[:len(x.split())//2]))
    df['Y'] = df[col_name].apply(lambda x: ' '.join(x.split()[len(x.split())//2:]))
    return df

spotifyDFMS3=split_sentences(spotifyDFMS3,"text")
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y
0,Bob Dylan,4Th Time Around,/b/bob+dylan/4th+time+around_20021421.html,"When she said, ""Don't waste your words, they'r...","When she said, ""Don't waste your words, they'r...","I waited in the hallway, she went to get it, A..."
1,Bob Dylan,A Satisfied Mind,/b/bob+dylan/a+satisfied+mind_20021501.html,How many times have you heard someone say \r\...,How many times have you heard someone say If I...,"suddenly it happened Hmm, I lost every dime Bu..."


In [45]:
print(len(spotifyDFMS3["Y"][0])+len(spotifyDFMS3["X"][0]))
print(len(spotifyDFMS3["text"][0]))

1338
1482


There is a difference in the values since the split_sentences method also removes the extra unneeded tokens such as white spaces adn \n.

In [46]:
spotifyDFMS3['artist_lyric']=spotifyDFMS3["artist"]+" || "+spotifyDFMS3["X"]

In [47]:
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y,artist_lyric
0,Bob Dylan,4Th Time Around,/b/bob+dylan/4th+time+around_20021421.html,"When she said, ""Don't waste your words, they'r...","When she said, ""Don't waste your words, they'r...","I waited in the hallway, she went to get it, A...","Bob Dylan || When she said, ""Don't waste your ..."
1,Bob Dylan,A Satisfied Mind,/b/bob+dylan/a+satisfied+mind_20021501.html,How many times have you heard someone say \r\...,How many times have you heard someone say If I...,"suddenly it happened Hmm, I lost every dime Bu...",Bob Dylan || How many times have you heard som...


In [48]:
def max_words(df, col_name):
    max_word_count = df[col_name].apply(lambda x: len(x.split(" "))).max()
    return max_word_count

max_word_count_X = max_words(spotifyDFMS3, 'X')
max_word_count_Y = max_words(spotifyDFMS3, 'Y')
max_word_count = max_words(spotifyDFMS3, 'artist_lyric')
print("X max :", max_word_count_X)
print("Y max :", max_word_count_Y)
print("X AND artist max:",max_word_count)

X max : 396
Y max : 397
X AND artist max: 399


In [49]:
from datasets import load_dataset
spotifyDataset = load_dataset('csv',data_files="/kaggle/input/smalldataset/spotifyDFMS3.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [50]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [51]:
def split_sentences(df,OldDF):
    x=[]
    y=[]
    artist_lyric=[]
    for string in OldDF["X"]:
        x.append(string)
    for string in OldDF["Y"]:
        y.append(string)
    for string in OldDF["artist_lyric"]:
        artist_lyric.append(string)
    df = df.add_column("X", x)
    df = df.add_column("Y", y)
    df = df.add_column("artist_lyric", artist_lyric)
    return df

spotifyDataset=split_sentences(spotifyDataset,spotifyDFMS3)

In [52]:
spotifyDataset[0]

{'artist': 'Bob Dylan',
 'song': '4Th Time Around',
 'link': '/b/bob+dylan/4th+time+around_20021421.html',
 'text': 'When she said, "Don\'t waste your words, they\'re just lies,"  \r\nI cried she was deaf.  \r\nAnd she worked on my face until breaking my eyes,  \r\nThen said, "What else you got left?"  \r\nIt was then that I got up to leave  \r\nBut she said, "Don\'t forget,  \r\nEverybody must give something back  \r\nFor something they get."  \r\n  \r\nI stood there and hummed,  \r\nI tapped on her drum  \r\nAnd asked her how come.  \r\nAnd she buttoned her boot,  \r\nAnd straightened her suit,  \r\nThen she said, "Don\'t get cute."  \r\nSo I forced my hands in my pockets  \r\nAnd felt with my thumbs,  \r\nAnd gallantly handed her  \r\nMy very last piece of gum.  \r\n  \r\nShe threw me outside,  \r\nI stood in the dirt where ev\'ryone walked.  \r\nAnd after finding I\'d forgotten my shirt,  \r\nI went back and knocked.  \r\nI waited in the hallway, she went to get it,  \r\nAnd I trie

In [53]:
columns_to_drop = ["song", "link","text"]
spotifyDataset = spotifyDataset.remove_columns(columns_to_drop)

In [54]:
spotifyDataset[0]

{'artist': 'Bob Dylan',
 'X': 'When she said, "Don\'t waste your words, they\'re just lies," I cried she was deaf. And she worked on my face until breaking my eyes, Then said, "What else you got left?" It was then that I got up to leave But she said, "Don\'t forget, Everybody must give something back For something they get." I stood there and hummed, I tapped on her drum And asked her how come. And she buttoned her boot, And straightened her suit, Then she said, "Don\'t get cute." So I forced my hands in my pockets And felt with my thumbs, And gallantly handed her My very last piece of gum. She threw me outside, I stood in the dirt where ev\'ryone walked. And after finding I\'d forgotten my shirt, I went back and knocked.',
 'Y': 'I waited in the hallway, she went to get it, And I tried to make sense Out of that picture of you in your wheelchair That leaned up against-- Her Jamaican rum And when she did come, I asked her for some. She said, "No, dear." I said, "Your words aren\'t clear

In [55]:
spotifyDataset

Dataset({
    features: ['artist', 'X', 'Y', 'artist_lyric'],
    num_rows: 943
})

In [56]:
def process_dataset(example):
    artist = example['artist']
    first_half = example['X']
    second_half = example['Y']
    
    input_text = f"{artist}: {first_half}"
    target_text = second_half
    
    return {'input_text': input_text, 'target_text': target_text}

spotifyDataset = spotifyDataset.map(process_dataset, remove_columns=["X", "Y", "artist_lyric"])

Map:   0%|          | 0/943 [00:00<?, ? examples/s]

In [57]:
spotifyDataset

Dataset({
    features: ['artist', 'input_text', 'target_text'],
    num_rows: 943
})

In [58]:
def tokenize_function(examples):
    inputs = [f"<s> {text} </s>" for text in examples['input_text']]
    targets = [f"<s> {text} </s>" for text in examples['target_text']]
    model_inputs = tokenizer(inputs, max_length=max_word_count, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_word_count, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    decoder_input_ids = labels['input_ids'].copy()
    decoder_start_token_id = tokenizer.pad_token_id
    
    for idx in range(len(decoder_input_ids)):
        decoder_input_ids[idx] = [decoder_start_token_id] + decoder_input_ids[idx][:-1]
    
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs

spotifyDataset = spotifyDataset.map(tokenize_function, batched=True, remove_columns=["artist", "input_text", "target_text"])

Map:   0%|          | 0/943 [00:00<?, ? examples/s]

In [59]:
spotifyDataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 943
})

In [60]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [61]:
spotifyDataset=spotifyDataset.train_test_split(test_size=0.2)

In [62]:
spotifyDataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 754
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 189
    })
})

In [63]:
import tensorflow as tf

In [64]:
train_dataset = spotifyDataset["train"].with_format("numpy")[:] 
test_dataset = spotifyDataset["test"].with_format("numpy")[:]

In [65]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [66]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage()  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=aafcc430904c821d473fdb816ab3a4124438b2e56acb39760f2afa59441ddfc6
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% | 96% |


In [67]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [68]:
model.fit(train_dataset, validation_data=test_dataset,batch_size=16, epochs=3)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
48/48 [==============================] - 164s 1s/step - loss: 4.4051 - sparse_categorical_accuracy: 0.4947 - val_loss: 1.2440 - val_sparse_categorical_accuracy: 0.7962
Epoch 2/3
48/48 [==============================] - 35s 728ms/step - loss: 1.6087 - sparse_categorical_accuracy: 0.7422 - val_loss: 1.1684 - val_sparse_categorical_accuracy: 0.8048
Epoch 3/3
48/48 [==============================] - 35s 729ms/step - loss: 1.2830 - sparse_categorical_accuracy: 0.7863 - val_loss: 1.1071 - val_sparse_categorical_accuracy: 0.8108


In [71]:
inputs = tokenizer(spotifyDFMS3["X"][1],return_tensors="tf").input_ids
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

"In reality it's so difficult to find One man in ten with a satisfied mind. Hmm, once I was wading in fortune and fame It's so hard to find"